# Visualize `AssumeRole` Graphs

In AWS, actors can assume roles, which can themselves assume roles, and so on. This can sometimes be a vector for lateral movement or privilege escalation. Here, we visualize `AssumeRole` chains to help us look for potential threat activity.

Install libraries.

In [ ]:
%pip install https://scanner-dev-public.s3.us-west-2.amazonaws.com/sdks/python/scanner_client-0.0.1-py3-none-any.whl

In [ ]:
%pip install yfiles_jupyter_graphs pandas

In [ ]:
import os
import pandas as pd
from scanner_client import Scanner
from yfiles_jupyter_graphs import GraphWidget
from datetime import datetime, timezone, timedelta

Add utility function to convert Scanner search results to a `pandas` data frame.

In [ ]:
def convert_results_to_data_frame(results):
    rows = [row.columns.to_dict() for row in results.rows]
    column_tags = results.column_tags.to_dict()
    if len(column_tags) > 0:
        # If this is a table, use the column ordering in the data frame
        return pd.DataFrame(data=rows, columns=results.column_ordering)
    else:
        # Otherwise, this is a list of log events, so use pandas JSON
        # normalization to set the table columns to the union of all keys.
        return pd.json_normalize(rows)


Initialize Scanner API client:

In [ ]:
scanner = Scanner(
    api_url=os.environ["SCANNER_API_URL"],
    api_key=os.environ["SCANNER_API_KEY"],
)

Set analyzed time range to be the last 7 days.

In [ ]:
end_time = datetime.now(tz=timezone.utc)
start_time = end_time - timedelta(days=7)

Query for all `AssumeRole` calls that have a target role ARN and were not made by AWS Service or Account.

In [ ]:
response = scanner.query.blocking_query(
    start_time=start_time.isoformat(),
    end_time=end_time.isoformat(),
    query_text="""
        %ingest.source_type: "aws:cloudtrail"
        eventName: "AssumeRole"
        requestParameters.roleArn: *
        (not userIdentity.type: ("AWSService" or "AWSAccount"))
        | rename 
          userIdentity.sessionContext.sessionIssuer.arn as sourceArn,
          requestParameters.roleArn as targetArn
        | stats
          min(eventTime) as firstTime,
          max(eventTime) as lastTime
          by
          sourceArn,
          targetArn
    """
)
len(response.results.rows)

In [ ]:
assume_role_df = convert_results_to_data_frame(response.results)

In [ ]:
assume_role_df.head()

Compute nodes and edges of `AssumeRole` operations.

In [ ]:
node_ids = set(assume_role_df['targetArn'].unique()).union(set(assume_role_df['sourceArn'].unique()))
nodes = []
for node_id in node_ids:
    if not node_id:
        continue
    parts = node_id.split('/')
    account_id = parts[0].split(':')[4]
    user_name = parts[1]
    nodes.append({
        'id': node_id,
        'properties': {
            'arn': node_id,
            'user_name': user_name,
            'account_id': account_id,
            'label': f"{user_name} ({account_id})",
        },
    })

edges = []
for i, row in enumerate(assume_role_df.to_dict(orient="records")):
    if not row['sourceArn'] or not row['targetArn']:
        next
    count = row.get('@q.count', 0)
    edges.append({
        'id': i,
        'start': row['sourceArn'],
        'end': row['targetArn'],
        'properties': {
            'source_arn': row['sourceArn'],
            'target_arn': row['targetArn'],
            'label': str(count),
            'count': count,
        }
    })


Generate interactive graph visualization of `AssumeRole` chains.
- Click and drag to navigate. Use mouse wheel to zoom in/out.
- Click on a node or an edge to select it and see what it is connected to.
- When a node or edge is selected, inspect its properties in the `Data` tab in the side bar.
- Search for a node or edge via the `Search` tab in the side bar.
- Change the layout of the graph to examine relationships in different ways.

In [ ]:
w = GraphWidget()
w.nodes = nodes
w.edges = edges
w.directed = True
w